In [1]:
import fitz, re
from pix2text import Pix2Text

[WARNING 2025-07-26 17:24:54,793 _showwarnmsg:109] d:\User\ProjectGithub\hiepnguyenn-99\RAG-Solve-Math\ragmathenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 


# giao_trinh_4.pdf

In [2]:
# trích suất từ file pdf sang txt

# kiểm tra giao nhau
def intersects(b1, b2):
    x0, y0, x1, y1 = b1
    X0, Y0, X1, Y1 = b2
    return not (x0 > X1 or y0 > Y1 or x1 < X0 or y1 < Y0)

# phát hiện công thức và chuyển thành latex
def extract_text_and_formula(page, header_cut, footer_cut, dpi, p2t):
    rect = page.rect
    clip = fitz.Rect(
        rect.x0,
        rect.y0 + header_cut,
        rect.x1,
        rect.y1 - footer_cut
    )
    layout = page.get_text(
        'dict',
        sort=True,
        clip=clip
        )
    pix = page.get_pixmap(dpi=dpi)
    img = pix.pil_image() # chuyển về ảnh pil
    zoom = dpi/72.0 # 1 point = 1/72 inch (pt->px)
    mat_forward = fitz.Matrix(zoom, zoom)
    mat_inverse = ~mat_forward # ma trận nghịch đảo (px->pt)

    # phát hiện công thức trong ảnh
    page_obj = p2t.recognize_page(
        img,
        file_type='text_formula',
        resized_shape=1024,
        return_text=False,
        save_debug_res=None
    )

    formulas = []
    if page_obj:
        formulas = [
            {
                'bbox_pt': fitz.Rect(f.box) * mat_inverse, # chuyển luôn về đơn vị point
                'latex': f.text
            }
            for f in page_obj.elements if f.type.name == "FORMULA"
        ]

    if not formulas:
        return layout
    
    seen_formulas = set()
    for block in layout.get('blocks', []):
        if block['type'] != 0: # chuỗi văn bản có type == 0
            continue
        
        for line in block.get('lines', []):
            line_bbox = fitz.Rect(line['bbox']) # pt
            relevant_formulas = [f for f in formulas if intersects(line_bbox, f['bbox_pt'])]
            if not relevant_formulas:
                continue

            new_spans = []
            original_spans = line['spans']
            # xác định span cần loại (cải thiện hiệu suất)
            spans_to_remove_idx = set()
            for i, span in enumerate(original_spans):
                sb = fitz.Rect(span['bbox'])
                for f in relevant_formulas:
                    r_pt = f['bbox_pt']
                    overlap = abs(sb & r_pt)
                    if overlap > 0.8 * abs(sb):
                        spans_to_remove_idx.add(i)
                        break

            for i, span in enumerate(original_spans):
                if i not in spans_to_remove_idx:
                    new_spans.append(span)

            for f in relevant_formulas:
                if f['latex'] not in seen_formulas:
                    new_spans.append({
                        'type': 'formula',
                        'text': f"$$ {f['latex']} $$",
                        'bbox': f['bbox_pt']
                    })
                    seen_formulas.add(f['latex'])
            line['spans'] = sorted(new_spans, key=lambda s: s['bbox'][0])
    return layout


In [ ]:
# chuyển txt sang md
def render_md(layout):
    md = ''
    subchapter = re.compile(r'^\d+\.\d+.*') # mục con của chương
    saw_chapter = False
    saw_subchapter = False
    skip = False

    for block in layout['blocks']:
        for line in block['lines']:
            spans = line['spans']
            line_text = ''
            for i, s in enumerate(spans):
                text = s['text']
                if i == 0:
                    line_text += text
                # xử lý dấu cách
                else:
                    pre = spans[i-1]['text']
                    if pre and pre[-1] != ' ' and text and text[0] != ' ':
                        line_text += ' ' + text
                    else:
                        line_text += text

            stripped = line_text.lstrip() # loại bỏ khoảng trắng bên trái

            # thêm dấu #
            if stripped.startswith('Chương'):
                saw_chapter = True
                continue
            elif subchapter.match(stripped):
                saw_subchapter = True
                continue

            if saw_chapter:                
                saw_chapter = False
                skip = True
                md += '# ' + stripped + '\n'
                continue
            
            if skip:
                if saw_subchapter:
                    skip = False
                    saw_subchapter = False
                    line_text = '## ' + stripped
                else:
                    continue
            else:
                if saw_subchapter:
                    line_text = '## ' + stripped
                    saw_subchapter = False
                elif stripped.startswith('Định nghĩa'):
                    line_text = '### ' + stripped

            md += line_text + '\n'
        md += '\n'
    return md

In [17]:
# run
providers = ['CUDAExecusionProvider', 'CPUExecutionProvider']

p2t = Pix2Text.from_config(
    analyzer_config={
        "model_name": "mfd-pro",
        "model_backend": "onnx",
        "providers": providers,
    },
    formula_ocr_config={
        "model_name": "mfr-pro",
        "model_backend": "onnx",
        "providers": providers,
    }
)

doc = fitz.open('source/giao_trinh_4.pdf')
page = doc.load_page(12)
header_cut = 40
footer_cut = 20
dpi=300
layout = extract_text_and_formula(page, header_cut, footer_cut, dpi, p2t)
md = render_md(layout)
print(md)

[INFO] 2025-07-26 23:22:44,086 [RapidOCR] base.py:24: Using engine_name: onnxruntime
[INFO] 2025-07-26 23:22:44,094 [RapidOCR] main.py:55: Using C:\Users\admin\AppData\Roaming\cnstd\1.2\ppocr\ch_PP-OCRv5_det\ch_PP-OCRv5_det_infer.onnx



0: 1024x736 3 titles, 13 plain texts, 3 abandons, 1 isolate_formula, 1267.7ms
Speed: 0.0ms preprocess, 1267.7ms inference, 0.9ms postprocess per image at shape (1, 3, 1024, 736)
Loading C:\Users\admin\AppData\Roaming\pix2text\1.1\mfd-1.5-onnx\pix2text-mfd-1.5.onnx for ONNX Runtime inference...
Using ONNX Runtime CPUExecutionProvider

WARNING imgsz=[200, 1024] must be multiple of max stride 32, updating to [224, 1024]
0: 224x1024 2 embeddings, 180.3ms
Speed: 2.5ms preprocess, 180.3ms inference, 1.1ms postprocess per image at shape (1, 3, 224, 1024)


100%|██████████| 2/2 [00:00<00:00,  4.30it/s]



WARNING imgsz=[254, 1024] must be multiple of max stride 32, updating to [256, 1024]
0: 256x1024 (no detections), 219.4ms
Speed: 1.7ms preprocess, 219.4ms inference, 1.2ms postprocess per image at shape (1, 3, 256, 1024)


0it [00:00, ?it/s]


...


WARNING imgsz=[281, 1024] must be multiple of max stride 32, updating to [288, 1024]
0: 288x1024 1 embedding, 2 isolateds, 218.8ms
Speed: 1.6ms preprocess, 218.8ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 1024)


100%|██████████| 1/1 [00:00<00:00,  3.08it/s]



WARNING imgsz=[1447, 1024] must be multiple of max stride 32, updating to [1472, 1024]
0: 1472x1024 5 embeddings, 1405.5ms
Speed: 14.4ms preprocess, 1405.5ms inference, 1.9ms postprocess per image at shape (1, 3, 1472, 1024)


100%|██████████| 5/5 [00:03<00:00,  1.55it/s]


Phần tửcơ sở của một hàng là phần tửkhác 0 đầu tiên
của hàng đó kểtừbên trái sang.
Hàng toàn số 0 thì không có phần tửcơ sở.

Ma trận bậc thang

1. Hàng toàn số 0 (nếu có) thì nằm dưới.

2. Phần từcơ sởhàng dưới nằm bên phải phần tửcơ sởhàng trên.

Ví dụ1.2









A =



 không phải bậc thang.







 không phải bậc thang.
B =



 

-2
1
0
− 1
0
0
0


 

2
0
0
0


 

-3



 

2
1
0
− 1
0
0


 

1
0
0


 

-1
0
2
0
0
0
0









C =



 là ma trận bậc thang.







 là ma trận bậc thang.
D =



 

1
2
0
1
0
0


 

-1
0
0
0
0


 

-4



 

2
1
0
0
2
0
0


 

3
2
0
0
0
0
0


 

-3
0
0
0
0
0

Ma trận chuyển vị
Chuyển vịcủa A = ( a ij ) m × n là ma trận A T = ( a ji ) n × m thu
được từ A bằng cách chuyển hàng thành cột.






−→ A T =



Ví dụ1.3 A =
 1
2
3
2
0
3


1
2
2
0
3
3

Ma trận vuông có sốhàng bằng sốcột.

Tập tất cảcác ma trận vuông trên trường số K được ký hiệu là M n [ K ] .

Đường chéo chính của ma trận vuông A

In [18]:
md = render_md(layout)
print(md)

Phần tửcơ sở của một hàng là phần tửkhác 0 đầu tiên
của hàng đó kểtừbên trái sang.
Hàng toàn số 0 thì không có phần tửcơ sở.

Ma trận bậc thang

1. Hàng toàn số 0 (nếu có) thì nằm dưới.

2. Phần từcơ sởhàng dưới nằm bên phải phần tửcơ sởhàng trên.

Ví dụ1.2









A =



 không phải bậc thang.







 không phải bậc thang.
B =



 

-2
1
0
− 1
0
0
0


 

2
0
0
0


 

-3



 

2
1
0
− 1
0
0


 

1
0
0


 

-1
0
2
0
0
0
0









C =



 là ma trận bậc thang.







 là ma trận bậc thang.
D =



 

1
2
0
1
0
0


 

-1
0
0
0
0


 

-4



 

2
1
0
0
2
0
0


 

3
2
0
0
0
0
0


 

-3
0
0
0
0
0

Ma trận chuyển vị
Chuyển vịcủa A = ( a ij ) m × n là ma trận A T = ( a ji ) n × m thu
được từ A bằng cách chuyển hàng thành cột.






−→ A T =



Ví dụ1.3 A =
 1
2
3
2
0
3


1
2
2
0
3
3

Ma trận vuông có sốhàng bằng sốcột.

Tập tất cảcác ma trận vuông trên trường số K được ký hiệu là M n [ K ] .

Đường chéo chính của ma trận vuông A

In [10]:
with open('extracted/giao_trinh_4.md', 'w', encoding='utf-8') as f:
    for line in md.splitlines():
        if line.strip():
            f.write(line + '\n')